# Plot temperature response over time

This notebook does the same as [2_compute_delta_T.ipynb](2_compute_delta_T.ipynb) except that it varies the ECS parameter and outputs a table of changes in temperature with respect to some reference year (defined below).  

In [22]:
from ar6_ch6_rcmipfigs.constants import BASE_DIR
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, INPUT_DATA_DIR, RESULTS_DIR

PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
PATH_DT_OUTPUT = RESULTS_DIR + '/tables/table_sens_dT_cs.csv'

**Output table found in:**

In [23]:
print(PATH_DT_OUTPUT)

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/results/tables/table_sens_dT_cs.csv


### General about computing $\Delta T$: 

We compute the change in GSAT temperature ($\Delta T$) from the effective radiative forcing (ERF) estimated from the RCMIP models (Nicholls et al 2020), by integrating with the impulse response function (IRF(t-t')) (Geoffroy at al 2013). See Nicholls et al (2020) for description of the RCMIP models and output. 

For any forcing agent $x$, with estimated ERF$_x$, the change in temperature $\Delta T$ is calculated as:


\begin{align*} 
\Delta T_x (t) &= \int_0^t ERF_x(t') IRF(t-t') dt' \\
\end{align*}

#### The Impulse response function (IRF):
In these calculations we use the impulse response function (Geoffroy et al 2013):
\begin{align*}
\text{IRF}(t)=& 0.885\cdot (\frac{0.587}{4.1}\cdot exp(\frac{-t}{4.1}) + \frac{0.413}{249} \cdot exp(\frac{-t}{249}))\\
\text{IRF}(t)= &  \frac{1}{\lambda}\sum_{i=1}^2\frac{a_i}{\tau_i}\cdot exp\big(\frac{-t}{\tau_i}\big) 
\end{align*}
with $\frac{1}{\lambda} = 0.885$ (K/Wm$^{-2}$), $a_1=0.587$, $\tau_1=4.1$(yr), $a_2=0.413$ and $\tau_2 = 249$(yr) (note that $i=1$ is the fast response and $i=2$ is the slow response and that $a_1+a_2=1$)


## Input data:
See [README.md](../../README.md)

# Code + figures

In [24]:
from ar6_ch6_rcmipfigs.constants import BASE_DIR
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, INPUT_DATA_DIR, RESULTS_DIR

PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
PATH_DT_OUTPUT = RESULTS_DIR + '/tables/table_sens_dT_cs.csv'

**Output table found in:**

In [25]:
print(PATH_DT_OUTPUT)

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/results/tables/table_sens_dT_cs.csv


## Imports:

In [26]:
import xarray as xr
from IPython.display import clear_output
import numpy as np
import os
import re
from pathlib import Path
import pandas as pd
import tqdm
from scmdata import df_append, ScmDataFrame
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:

climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'

## Set values:

ECS parameters:

In [28]:
ECS2ecsf = {'ECS = 2K':0.526, 'ECS = 3.4K':0.884, 'ECS = 5K': 1.136 }

Year to integrate from and to:

In [29]:
first_y ='1850'
last_y = '2100'

**Set reference year for temperature change:**

In [30]:
ref_year = '2021'

**Years to output change in**

In [31]:
years= ['2040', '2100']

## IRF:

In [32]:

def IRF(t, l=0.885, alpha1=0.587 / 4.1, alpha2=0.413 / 249, tau1=4.1, tau2=249):
    """
    Returns the IRF function for:
    :param t: Time in years
    :param l: climate sensitivity factor
    :param alpha1:
    :param alpha2:
    :param tau1:
    :param tau2:
    :return:
    IRF
    """
    return l * (alpha1 * np.exp(-t / tau1) + alpha2 * np.exp(-t / tau2))

## ERF:
Read ERF from file

### Define variables to look at:

In [33]:
# variables to plot:
variables_erf_comp = [
    'Effective Radiative Forcing|Anthropogenic|CH4',
    'Effective Radiative Forcing|Anthropogenic|Aerosols',
    'Effective Radiative Forcing|Anthropogenic|Tropospheric Ozone',
    'Effective Radiative Forcing|Anthropogenic|F-Gases|HFC',
    'Effective Radiative Forcing|Anthropogenic|Other|BC on Snow']
# total ERFs for anthropogenic and total:
variables_erf_tot = ['Effective Radiative Forcing|Anthropogenic',
                     'Effective Radiative Forcing']
# Scenarios to plot:
scenarios_fl = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp370-lowNTCF-aerchemmip',  # 'ssp370-lowNTCF', Due to mistake here
                'ssp585', 'historical']

### Open dataset:

In [34]:
ds = xr.open_dataset(PATH_DATASET)

## Integrate:
The code below integrates the read in ERFs with the pre defined impulse response function (IRF).

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

In [35]:
name_deltaT = 'Delta T'


def new_varname(var, nname):
    """
    var:str
        Old variable of format varname|bla|bla
    nname:str
        name for the resulting variable, based on var
    Returns
    -------
    new variable name with nname|bla|bla
    """
    return nname + '|' + '|'.join(var.split('|')[1:])


def integrate_(i, var, nvar, ds, ds_DT, csfac=0.885):
    """
    
    Parameters
    ----------
    i:int
        the index for the integral
    var:str
        the name of the EFR variables to integrate
    nvar:str
        the name of output integrated value

    ds:xr.Dataset
        the ds with the intput data
    ds_DT: xr.Dataset
        the ouptut ds with the integrated results
    csfac: climate sensitivity factor (for IRF)
    Returns
    -------
    None
    
    """
    # lets create a ds that goes from 0 to i inclusive
    ds_short = ds[{'time': slice(0, i + 1)}].copy()
    # lets get the current year
    current_year = ds_short['time'][{'time': i}].dt.year
    # lets get a list of years
    years = ds_short['time'].dt.year
    # lets get the year delta until current year(i)
    ds_short['end_year_delta'] = current_year - years

    # lets get the irf values from 0 until i
    ds_short['irf'] = IRF(
        ds_short['end_year_delta'] * ds_short['delta_t'], l=csfac
    )

    # lets do the famous integral
    ds_short['to_integrate'] = \
        ds_short[var] * \
        ds_short['irf'] * \
        ds_short['delta_t']

    # lets sum all the values up until i and set
    # this value at ds_DT
    # If whole array is null, set value to nan
    if np.all(ds_short['to_integrate'].isnull()):  # or last_null:
        _val = np.nan
    else:
        # 

        _ds_int = ds_short['to_integrate'].sum(['time'])
        # mask where last value is null (in order to not get intgral 
        _ds_m1 = ds_short['to_integrate'].isel(time=-1)
        # where no forcing data)
        _val = _ds_int.where(_ds_m1.notnull())
    # set value in dataframe:
    ds_DT[nvar][{'time': i}] = _val


def integrate_to_dT(ds, from_t, to_t, variables, csfac=0.885):
    """
    Integrate forcing to temperature change.

    :param ds: dataset containing the focings
    :param from_t: start time
    :param to_t: end time
    :param variables: variables to integrate
    :param csfac: climate sensitivity factor
    :return:
    """
    # slice dataset
    ds_sl = ds.sel(time=slice(from_t, to_t))
    len_time = len(ds_sl['time'])
    # lets create a result DS
    ds_DT = ds_sl.copy()

    # lets define the vars of the ds
    vars = variables  # variables_erf_comp+ variables_erf_tot #['EFR']
    for var in variables:
        namevar = new_varname(var, name_deltaT)
        # set all values to zero for results dataarray:
        ds_DT[namevar] = ds_DT[var] * 0
        # Units Kelvin:
        ds_DT[namevar].attrs['unit'] = 'K'
        if 'unit' in ds_DT[namevar].coords:
            ds_DT[namevar].coords['unit'] = 'K'

    for i in range(len_time):
        # da = ds[var]
        if (i % 20) == 0:
            print('%s of %s done' % (i, len_time))
        for var in variables:
            namevar = new_varname(var, name_deltaT)  # 'Delta T|' + '|'.join(var.split('|')[1:])

            # print(var)
            integrate_(i, var, namevar, ds_sl, ds_DT, csfac=csfac)
    clear_output()

    fname = 'DT_%s-%s.nc' % (from_t, to_t)
    # save dataset.
    ds_DT.to_netcdf(fname)
    return ds_DT



## Compute $\Delta T$ with 3 different climate sensitivities

In [36]:
dic_ds = {}
for key  in ECS2ecsf:
    dic_ds[key] = integrate_to_dT(ds, first_y, last_y,(variables_erf_comp+variables_erf_tot), csfac=ECS2ecsf[key])

## Table

### Setup table:

In [37]:

iterables = [list(ECS2ecsf.keys()), years]

def setup_table(scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    table = pd.DataFrame(columns=[var.split('|')[-1] for var in variables_erf_comp], index = _i).transpose()
    table.index.name=scenario_n
    return table


In [51]:
# Dicitonary of tables with different ESC:
scntab_dic = {}
for scn in scenarios_fl: 
    # Loop over scenrarios
    tab = setup_table(scenario_n=scn) # make table
    for var in variables_erf_comp:
        # Loop over variables
        tabvar = var.split('|')[-1]
        dtvar = new_varname(var, name_deltaT)
        for key in ECS2ecsf:
            # Loop over ESC parameters
            for year in years: 
                _tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                tab.loc[tabvar,key][year]=_tab_da.squeeze().mean('climatemodel').values
    scntab_dic[scn]=tab.copy()


/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWa

In [52]:
from IPython.display import display

for key in scntab_dic:
    display(scntab_dic[key])

ECS = 2K                          \
                                     2040                    2100   
ssp119                                                              
CH4                  -0.02659828770048701     -0.1296912754447551   
Aerosols              0.15646387941855683      0.2249195956185493   
Tropospheric Ozone   -0.04992508208229368    -0.10048092297791049   
HFC                  0.003468253587250494  -0.0029761684916363817   
BC on Snow          -0.011329101981495735    -0.01697901215240332   

                               ECS = 3.4K                         \
                                     2040                   2100   
ssp119                                                             
CH4                  -0.04470130480462069    -0.2179602423824401   
Aerosols              0.26295450457415265    0.37800175385322743   
Tropospheric Ozone   -0.08390451057176351   -0.16886907968150738   
HFC                  0.005828775990740371  -0.005001773662750117   
BC on Snow          -0.019039783558255186   -0.02853506985308847   

                                 ECS = 5K                         
                                     2040                   2100  
ssp119                                                            
CH4                  -0.05744421069915064    -0.2800937051430452  
Aerosols               0.3379143859685943     0.4857579099290342  
Tropospheric Ozone   -0.10782299096099926    -0.2170082290929777  
HFC                 0.0074903727663812964  -0.006427618643534087  
BC on Snow          -0.024467414165359597   -0.03666950153066572

ECS = 2K                        \
                                     2040                  2100   
ssp126                                                            
CH4                 -0.017367724889572122   -0.1231558283831311   
Aerosols              0.12215597107108145   0.22774103852286798   
Tropospheric Ozone  -0.033763352537327304  -0.09404727045262026   
HFC                  0.011032221399586888  -0.00132423401564886   
BC on Snow          -0.009436533713336538  -0.01532641975381759   

                              ECS = 3.4K                         \
                                    2040                   2100   
ssp126                                                            
CH4                 -0.02918834373076379   -0.20697671538153598   
Aerosols              0.2052963468190799     0.3827434943996489   
Tropospheric Ozone   -0.0567429727053181   -0.15805662942987894   
HFC                 0.018540843568887466  -0.002225518763942189   
BC on Snow           -0.0158591174954173  -0.025757709244058458   

                                 ECS = 5K                          
                                     2040                    2100  
ssp126                                                             
CH4                  -0.03750900280333449     -0.2659791274586253  
Aerosols               0.2638197398037043      0.4918513683687795  
Tropospheric Ozone   -0.07291857125932286    -0.20311349664292128  
HFC                  0.023826242414316925  -0.0028599426649754825  
BC on Snow          -0.020380042392300957   -0.033100404639423535

ECS = 2K                         \
                                     2040                   2100   
ssp245                                                             
CH4                  0.028937393882524302  -0.004182704112309194   
Aerosols              0.03637900641138267     0.1367591718072312   
Tropospheric Ozone  0.0009328311281484147  -0.039034660403500676   
HFC                  0.018234694544699594   0.038451376778719744   
BC on Snow          -0.003402782920874044  -0.012488796472324282   

                               ECS = 3.4K                         \
                                     2040                   2100   
ssp245                                                             
CH4                   0.04863242622082048  -0.007029487519546129   
Aerosols              0.06113886248604996    0.22983860813230494   
Tropospheric Ozone  0.0015677237971163881   -0.06560197679979962   
HFC                  0.030645380185388666    0.06462170546081415   
BC on Snow          -0.005718745441164742  -0.020988775820408107   

                                 ECS = 5K                         
                                     2040                   2100  
ssp245                                                            
CH4                   0.06249596853716284  -0.009033368577154454  
Aerosols              0.07856758799112298     0.2953582113555412  
Tropospheric Ozone   0.002014631485887075   -0.08430299281060225  
HFC                  0.039381393541404436    0.08304327760575213  
BC on Snow          -0.007348976042039757  -0.026972001506768793

ECS = 2K                         \
                                     2040                   2100   
ssp370                                                             
CH4                   0.05457162376258964    0.21565799113002856   
Aerosols            -0.011068885443324872  -0.007421477076560101   
Tropospheric Ozone   0.024602751780866523    0.06349719049159727   
HFC                   0.01704878795819055    0.06493327819133399   
BC on Snow          0.0036842324966297967   0.006754058664053657   

                               ECS = 3.4K                         \
                                     2040                   2100   
ssp370                                                             
CH4                    0.0917135273880785    0.36243662387632175   
Aerosols            -0.018602461467488916  -0.012472596455663742   
Tropospheric Ozone   0.041347590445410684      0.106713909495384   
HFC                  0.028652335655970435    0.10912741049646246   
BC on Snow           0.006191751952510926    0.01135092748863771   

                                ECS = 5K                        
                                    2040                  2100  
ssp370                                                          
CH4                  0.11785810759372976   0.46575566145192465  
Aerosols            -0.02390542559623008  -0.01602813300184842  
Tropospheric Ozone   0.05313446011989424   0.13713461672709976  
HFC                  0.03682019604658643   0.14023612932577073  
BC on Snow          0.007956821513633937  0.014586712247842122

ECS = 2K                        \
                                             2040                  2100   
ssp370-lowNTCF-aerchemmip                                                 
CH4                           0.05457162366753363      0.21565799026157   
Aerosols                      0.06273291382163072   0.14244876622095556   
Tropospheric Ozone         -0.0032062710739075963  0.012046389288997983   
HFC                           0.01704878793732071    0.0649332767375719   
BC on Snow                 -0.0037433274391409396  -0.00704176174526431   

                                      ECS = 3.4K                         \
                                            2040                   2100   
ssp370-lowNTCF-aerchemmip                                                 
CH4                          0.09171352722832657     0.3624366224167831   
Aerosols                     0.10542945973064936    0.23940058809757553   
Tropospheric Ozone         -0.005388485987327574    0.02024526260736541   
HFC                           0.0286523356208964    0.10912740805325775   
BC on Snow                 -0.006291067407225456  -0.011834443693562069   

                                        ECS = 5K                         
                                            2040                   2100  
ssp370-lowNTCF-aerchemmip                                                
CH4                          0.11785810738843767     0.4657556595763185  
Aerosols                     0.13548401159956747    0.30764600461407887  
Tropospheric Ozone         -0.006924570228058991   0.026016536563311144  
HFC                          0.03682019600151392    0.14023612618608688  
BC on Snow                 -0.008084448613810082  -0.015208063389011895

ECS = 2K                          \
                                      2040                    2100   
ssp585                                                               
CH4                   0.044123353916243016     0.11489216246985463   
Aerosols              0.047101148762349374      0.0979238452792276   
Tropospheric Ozone     0.01531662537912159    0.011390724974023542   
HFC                    0.02710784418483706     0.14984300008060603   
BC on Snow          -0.0018159626749880552  -0.0035326595202287737   

                                ECS = 3.4K                         \
                                      2040                   2100   
ssp585                                                              
CH4                    0.07415407768433238    0.19308872932196103   
Aerosols               0.07915858461200927    0.16457163351109735   
Tropospheric Ozone    0.025741248736014292    0.01914334767497496   
HFC                    0.04555766969466912     0.2518273993750109   
BC on Snow          -0.0030519220621472187  -0.005937017140460517   

                                 ECS = 5K                          
                                     2040                    2100  
ssp585                                                             
CH4                   0.09529302290656273     0.24813212274858323  
Aerosols              0.10172415398104348     0.21148571908213415  
Tropospheric Ozone    0.03307925176935772    0.024600501084583216  
HFC                    0.0585446977071766      0.3236153005543126  
BC on Snow          -0.003921926993890548  -0.0076294699904560365

ECS = 2K                         \
                                     2040                   2100   
historical                                                         
CH4                 0.0077132387149825565    0.02703927122654534   
Aerosols            -0.009908627912094015   -0.04251408931871051   
Tropospheric Ozone   0.005241636725860588    0.01849257773823193   
HFC                 0.0007444353141619498  0.0017742750627926067   
BC on Snow                            nan                    nan   

                               ECS = 3.4K                         \
                                     2040                   2100   
historical                                                         
CH4                  0.012962933505788199   0.045442425407349996   
Aerosols            -0.016652522954926197    -0.0714495341401904   
Tropospheric Ozone   0.008809138527872185   0.031078780837636932   
HFC                 0.0012511042161961299  0.0029818615123738887   
BC on Snow                            nan                    nan   

                                 ECS = 5K                         
                                     2040                   2100  
historical                                                        
CH4                  0.016658249392053515      0.058396600975961  
Aerosols             -0.02139962225881925   -0.09181750088603652  
Tropospheric Ozone   0.011320340913645699    0.03993834279587727  
HFC                 0.0016077538343877845  0.0038318944321908757  
BC on Snow                            nan                    nan

### Make table with all scenarios:

In [40]:
iterables = [list(ECS2ecsf.keys()), years]
iterables2 = [scenarios_fl, [var.split('|')[-1] for var in variables_erf_comp]]

def setup_table2():#scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    _r = pd.MultiIndex.from_product(iterables2, names=['', ''])
    
    table = pd.DataFrame(columns=_r, index = _i).transpose()
    return table



In [55]:
tab = setup_table2()#scenario_n=scn)

for scn in scenarios_fl:
    for var in variables_erf_comp:
        tabvar = var.split('|')[-1]
        dtvar = new_varname(var,name_deltaT)
        print(dtvar)
        for key in ECS2ecsf:
            for year in years: 
                # compute difference between year and ref year
                _da_y = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))#.squeeze()
                _da_refy = dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                #_tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                _tab_da = _da_y - _da_refy

                tab.loc[(scn, tabvar), (key,year)] =_tab_da.squeeze().mean('climatemodel').values#[0]



Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthropogenic|Other|BC on Snow
Delta T|Anthropogenic|CH4
Delta T|Anthropogenic|Aerosols
Delta T|Anthropogenic|Tropospheric Ozone
Delta T|Anthropogenic|F-Gases|HFC
Delta T|Anthr

/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/sarambl/miniconda3/envs/rcmip_ipcc/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWa

In [56]:
tab

ECS = 2K  \
                                                                2040   
                                                                       
ssp119                    CH4                   -0.02659828770048701   
                          Aerosols               0.15646387941855683   
                          Tropospheric Ozone    -0.04992508208229368   
                          HFC                   0.003468253587250494   
                          BC on Snow           -0.011329101981495735   
ssp126                    CH4                  -0.017367724889572122   
                          Aerosols               0.12215597107108145   
                          Tropospheric Ozone   -0.033763352537327304   
                          HFC                   0.011032221399586888   
                          BC on Snow           -0.009436533713336538   
ssp245                    CH4                   0.028937393882524302   
                          Aerosols               0.03637900641138267   
                          Tropospheric Ozone   0.0009328311281484147   
                          HFC                   0.018234694544699594   
                          BC on Snow           -0.003402782920874044   
ssp370                    CH4                    0.05457162376258964   
                          Aerosols             -0.011068885443324872   
                          Tropospheric Ozone    0.024602751780866523   
                          HFC                    0.01704878795819055   
                          BC on Snow           0.0036842324966297967   
ssp370-lowNTCF-aerchemmip CH4                    0.05457162366753363   
                          Aerosols               0.06273291382163072   
                          Tropospheric Ozone  -0.0032062710739075963   
                          HFC                    0.01704878793732071   
                          BC on Snow          -0.0037433274391409396   
ssp585                    CH4                   0.044123353916243016   
                          Aerosols              0.047101148762349374   
                          Tropospheric Ozone     0.01531662537912159   
                          HFC                    0.02710784418483706   
                          BC on Snow          -0.0018159626749880552   
historical                CH4                  0.0077132387149825565   
                          Aerosols             -0.009908627912094015   
                          Tropospheric Ozone    0.005241636725860588   
                          HFC                  0.0007444353141619498   
                          BC on Snow                             nan   

                                                                      \
                                                                2100   
                                                                       
ssp119                    CH4                    -0.1296912754447551   
                          Aerosols                0.2249195956185493   
                          Tropospheric Ozone    -0.10048092297791049   
                          HFC                 -0.0029761684916363817   
                          BC on Snow            -0.01697901215240332   
ssp126                    CH4                    -0.1231558283831311   
                          Aerosols               0.22774103852286798   
                          Tropospheric Ozone    -0.09404727045262026   
                          HFC                   -0.00132423401564886   
                          BC on Snow            -0.01532641975381759   
ssp245                    CH4                  -0.004182704112309194   
                          Aerosols                0.1367591718072312   
                          Tropospheric Ozone   -0.039034660403500676   
                          HFC                   0.038451376778719744   
                          BC on Snow           -0.012488796472324282   
ssp370                    C

## Save output

In [57]:
tab.to_csv(PATH_DT_OUTPUT)